<a href="https://colab.research.google.com/github/laynz28/RCV-AI-COVER-ALL/blob/main/RVC_AI_COVER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Preparation

In [ ]:
#@title Test Runtime
!nvidia-smi
!nvcc -V
!free -h

In [ ]:
#@title Installation Dependencies
!pip install yt_dlp
!pip install ffmpeg
!mkdir youtubeaudio
!python3 -m pip install -U demucs
!pip install yt_dlp
!pip install ffmpeg
!apt-get -y install build-essential python3-dev ffmpeg
!pip3 install --upgrade setuptools wheel
!pip3 install --upgrade pip
!pip3 install faiss-gpu fairseq gradio ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2
!apt -y install -qq aria2
!pip install mega.py --quiet
!pip install gdown --quiet
import ipywidgets as widgets
from IPython.display import clear_output
import os, shutil
success=widgets.Button(description="\u2714 Done",
disabled=True, button_style="success")

In [ ]:
#@title Mount Drive

from google.colab import drive
drive.mount('/content/drive')

# 2.  download wav audio from YOUTUBE and SEPARATE VOCAL & MUSIC

In [ ]:
#@title Start Separate Vocal and Instrument using Demucs
#@markdown This program is standart separating Vocal and Instrument.
#@markdown Use webUI for more feature.
import subprocess
input_file = "" #@param {type:"string"}
OUTPUT_DIR = "/content/separated"

command = f"demucs --two-stems=vocals {input_file}"
result = subprocess.run(command.split(), stdout=subprocess.PIPE)
print(result.stdout.decode())
#input_file = input("Masukkan file audio: ")

# 3. Install to Google Drive

In [ ]:
#@title Install to Google Drive (for Resuming Training & Automatic Saving)
#@markdown <small> This Notebook is based on another found in: https://github.com/ardha27/AI-Song-Cover-RVC < Visit this repo to read more and support.
%cd /content
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import clear_output
from ipywidgets import Button
import os
if not os.path.exists('/content/drive'):
    print("Your drive is not mounted. Creating Fake Drive.")
    os.makedirs('/content/drive/MyDrive')
source = "Rejekts"
!wget https://huggingface.co/{source}/project/resolve/main/project-main.zip -O '/content/project-main.zip' && unzip -n 'project-main.zip' -d /content/drive/MyDrive
!cd '/content/drive/MyDrive/project-main' && python download_files.py && pip install -r 'requirements-safe.txt'
!pip install pyngrok tensorflow==2.12.0
!rm /content/project-main.zip
!rm -r /content/sample_data
!mkdir -p /content/dataset
clear_output()
Button(description="\u2714 Success", button_style="success")

#make dataset

In [ ]:
 #@title Input Form

Mode = "Splitting" #@param ["Separate", "Splitting"]
dataset = "Youtube" #@param ["Youtube", "Drive"]
url = "your YouTube URL" #@param {type:"string"}
drive_path = "" #@param {type:"string"}
#@markdown the audio name must be the same as the patch drive name if using Google Drive
AUDIO_NAME = "" #@param {type:"string"}

In [ ]:
#@title Download Youtube WAV
from __future__ import unicode_literals

if dataset == "Drive":
  print("Dataset is set to Drive. Skipping this section")
elif dataset == "Youtube":
  import yt_dlp
  import ffmpeg
  import sys


  ydl_opts = {
      'format': 'bestaudio/best',
  #    'outtmpl': 'output.%(ext)s',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'wav',
      }],
      "outtmpl": f'youtubeaudio/{AUDIO_NAME}',  # this is where you can edit how you'd like the filenames to be formatted
  }
  def download_from_url(url):
      ydl.download([url])
      # stream = ffmpeg.input('output.m4a')
      # stream = ffmpeg.output(stream, 'output.wav')


  with yt_dlp.YoutubeDL(ydl_opts) as ydl:

        download_from_url(url)

In [ ]:
#@title 2. Install Demucs for Separating Audio
!python3 -m pip install -U demucs

In [ ]:
#@title Separate Vocal and Instrument/Noise using Demucs
import subprocess
AUDIO_INPUT = f"/content/youtubeaudio/{AUDIO_NAME}.wav"

if dataset == "Drive":
  command = f"demucs --two-stems=vocals {drive_path}"
elif dataset == "Youtube":
  command = f"demucs --two-stems=vocals {AUDIO_INPUT}"

result = subprocess.run(command.split(), stdout=subprocess.PIPE)
print(result.stdout.decode())

In [ ]:
!mkdir -p /content/drive/MyDrive/audio/{AUDIO_NAME}
!cp -r /content/separated/htdemucs/{AUDIO_NAME}/* /content/drive/MyDrive/audio/{AUDIO_NAME}
if dataset == "Youtube":
  !cp -r /content/youtubeaudio/{AUDIO_NAME}.wav /content/drive/MyDrive/audio/{AUDIO_NAME}

In [ ]:
#@title 3. Split The Audio into Smaller Duration Before Training
if Mode == "Separate":
    print("Mode is set to Separate. Skipping this section")
elif Mode ==  "Splitting":
  !pip install numpy
  !pip install librosa
  !pip install soundfile
  !mkdir -p dataset/{AUDIO_NAME}



import numpy as np
import librosa
import soundfile


# This function is obtained from librosa.
def get_rms(
    y,
    *,
    frame_length=2048,
    hop_length=512,
    pad_mode="constant",
):
    padding = (int(frame_length // 2), int(frame_length // 2))
    y = np.pad(y, padding, mode=pad_mode)

    axis = -1
    # put our new within-frame axis at the end for now
    out_strides = y.strides + tuple([y.strides[axis]])
    # Reduce the shape on the framing axis
    x_shape_trimmed = list(y.shape)
    x_shape_trimmed[axis] -= frame_length - 1
    out_shape = tuple(x_shape_trimmed) + tuple([frame_length])
    xw = np.lib.stride_tricks.as_strided(
        y, shape=out_shape, strides=out_strides
    )
    if axis < 0:
        target_axis = axis - 1
    else:
        target_axis = axis + 1
    xw = np.moveaxis(xw, -1, target_axis)
    # Downsample along the target axis
    slices = [slice(None)] * xw.ndim
    slices[axis] = slice(0, None, hop_length)
    x = xw[tuple(slices)]

    # Calculate power
    power = np.mean(np.abs(x) ** 2, axis=-2, keepdims=True)

    return np.sqrt(power)


class Slicer:
    def __init__(self,
                 sr: int,
                 threshold: float = -40.,
                 min_length: int = 5000,
                 min_interval: int = 300,
                 hop_size: int = 20,
                 max_sil_kept: int = 5000):
        if not min_length >= min_interval >= hop_size:
            raise ValueError('The following condition must be satisfied: min_length >= min_interval >= hop_size')
        if not max_sil_kept >= hop_size:
            raise ValueError('The following condition must be satisfied: max_sil_kept >= hop_size')
        min_interval = sr * min_interval / 1000
        self.threshold = 10 ** (threshold / 20.)
        self.hop_size = round(sr * hop_size / 1000)
        self.win_size = min(round(min_interval), 4 * self.hop_size)
        self.min_length = round(sr * min_length / 1000 / self.hop_size)
        self.min_interval = round(min_interval / self.hop_size)
        self.max_sil_kept = round(sr * max_sil_kept / 1000 / self.hop_size)

    def _apply_slice(self, waveform, begin, end):
        if len(waveform.shape) > 1:
            return waveform[:, begin * self.hop_size: min(waveform.shape[1], end * self.hop_size)]
        else:
            return waveform[begin * self.hop_size: min(waveform.shape[0], end * self.hop_size)]

    def slice(self, waveform):
        if len(waveform.shape) > 1:
            samples = waveform.mean(axis=0)
        else:
            samples = waveform
        if samples.shape[0] <= self.min_length:
            return [waveform]
        rms_list = get_rms(y=samples, frame_length=self.win_size, hop_length=self.hop_size).squeeze(0)
        sil_tags = []
        silence_start = None
        clip_start = 0
        for i, rms in enumerate(rms_list):
            # Keep looping while frame is silent.
            if rms < self.threshold:
                # Record start of silent frames.
                if silence_start is None:
                    silence_start = i
                continue
            # Keep looping while frame is not silent and silence start has not been recorded.
            if silence_start is None:
                continue
            # Clear recorded silence start if interval is not enough or clip is too short
            is_leading_silence = silence_start == 0 and i > self.max_sil_kept
            need_slice_middle = i - silence_start >= self.min_interval and i - clip_start >= self.min_length
            if not is_leading_silence and not need_slice_middle:
                silence_start = None
                continue
            # Need slicing. Record the range of silent frames to be removed.
            if i - silence_start <= self.max_sil_kept:
                pos = rms_list[silence_start: i + 1].argmin() + silence_start
                if silence_start == 0:
                    sil_tags.append((0, pos))
                else:
                    sil_tags.append((pos, pos))
                clip_start = pos
            elif i - silence_start <= self.max_sil_kept * 2:
                pos = rms_list[i - self.max_sil_kept: silence_start + self.max_sil_kept + 1].argmin()
                pos += i - self.max_sil_kept
                pos_l = rms_list[silence_start: silence_start + self.max_sil_kept + 1].argmin() + silence_start
                pos_r = rms_list[i - self.max_sil_kept: i + 1].argmin() + i - self.max_sil_kept
                if silence_start == 0:
                    sil_tags.append((0, pos_r))
                    clip_start = pos_r
                else:
                    sil_tags.append((min(pos_l, pos), max(pos_r, pos)))
                    clip_start = max(pos_r, pos)
            else:
                pos_l = rms_list[silence_start: silence_start + self.max_sil_kept + 1].argmin() + silence_start
                pos_r = rms_list[i - self.max_sil_kept: i + 1].argmin() + i - self.max_sil_kept
                if silence_start == 0:
                    sil_tags.append((0, pos_r))
                else:
                    sil_tags.append((pos_l, pos_r))
                clip_start = pos_r
            silence_start = None
        # Deal with trailing silence.
        total_frames = rms_list.shape[0]
        if silence_start is not None and total_frames - silence_start >= self.min_interval:
            silence_end = min(total_frames, silence_start + self.max_sil_kept)
            pos = rms_list[silence_start: silence_end + 1].argmin() + silence_start
            sil_tags.append((pos, total_frames + 1))
        # Apply and return slices.
        if len(sil_tags) == 0:
            return [waveform]
        else:
            chunks = []
            if sil_tags[0][0] > 0:
                chunks.append(self._apply_slice(waveform, 0, sil_tags[0][0]))
            for i in range(len(sil_tags) - 1):
                chunks.append(self._apply_slice(waveform, sil_tags[i][1], sil_tags[i + 1][0]))
            if sil_tags[-1][1] < total_frames:
                chunks.append(self._apply_slice(waveform, sil_tags[-1][1], total_frames))
            return chunks

if Mode == "Separate":
    print("Mode is set to Separate. Skipping this section")

elif Mode ==  "Splitting":
  audio, sr = librosa.load(f'/content/separated/htdemucs/{AUDIO_NAME}/vocals.wav', sr=None, mono=False)  # Load an audio file with librosa.
  slicer = Slicer(
      sr=sr,
      threshold=-40,
      min_length=5000,
      min_interval=500,
      hop_size=10,
      max_sil_kept=500
  )
  chunks = slicer.slice(audio)
  for i, chunk in enumerate(chunks):
      if len(chunk.shape) > 1:
          chunk = chunk.T  # Swap axes if the audio is stereo.
      soundfile.write(f'/content/dataset/{AUDIO_NAME}/split_{i}.wav', chunk, sr)  # Save sliced audio files with soundfile.

In [ ]:
if Mode == "Separate":
    print("Mode is set to Separate. Skipping this section")
elif Mode ==  "Splitting":
  !mkdir -p /content/drive/MyDrive/dataset/{AUDIO_NAME}
  !cp -r /content/dataset/{AUDIO_NAME}/* /content/drive/MyDrive/dataset/{AUDIO_NAME}

# DOWNLOAD YOUTUBE WAV

In [ ]:
#@title Separate vocal if you want
Mode = "Separate" #@param ["Separate"]
url = "" #@param {type:"string"}
drive_path = "" #@param {type:"string"}
AUDIO_NAME = "" #@param {type:"string"}

In [ ]:
#@title Download Youtube WAV
from __future__ import unicode_literals

if dataset == "Drive":
  print("Dataset is set to Drive. Skipping this section")
elif dataset == "Youtube":
  import yt_dlp
  import ffmpeg
  import sys


  ydl_opts = {
      'format': 'bestaudio/best',
  #    'outtmpl': 'output.%(ext)s',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'wav',
      }],
      "outtmpl": f'youtubeaudio/{AUDIO_NAME}',  # this is where you can edit how you'd like the filenames to be formatted
  }
  def download_from_url(url):
      ydl.download([url])
      # stream = ffmpeg.input('output.m4a')
      # stream = ffmpeg.output(stream, 'output.wav')


  with yt_dlp.YoutubeDL(ydl_opts) as ydl:

        download_from_url(url)

In [ ]:
#@title Separate Vocal and Instrument/Noise using Demucs
import subprocess
AUDIO_INPUT = f"/content/youtubeaudio/{AUDIO_NAME}.wav"

if dataset == "Drive":
  command = f"demucs --two-stems=vocals {drive_path}"
elif dataset == "Youtube":
  command = f"demucs --two-stems=vocals {AUDIO_INPUT}"

result = subprocess.run(command.split(), stdout=subprocess.PIPE)
print(result.stdout.decode())

In [ ]:
!mkdir -p /content/drive/MyDrive/audio/{AUDIO_NAME}
!cp -r /content/separated/htdemucs/{AUDIO_NAME}/* /content/drive/MyDrive/audio/{AUDIO_NAME}
if dataset == "Youtube":
  !cp -r /content/youtubeaudio/{AUDIO_NAME}.wav /content/drive/MyDrive/audio/{AUDIO_NAME}

#  Download models

NAME = GIVE A NAME FOR MODELS


URL = YOUR VOICE MODELS URL FORM HUGGINGFACE MEGA DRIVE

In [ ]:
#@title Download The Model
from mega import Mega
import os
MODEL = "LYN"  #@param {type:"string"}
MODELEPOCH = ''
url='https://huggingface.co/spaces/LaynzKunz/RANDDOM_RCV/resolve/main/Laynz.zip'#@param {type:"string"}
MODELZIP = MODEL + '.zip'
modelname_path='/content/zips/'+MODELZIP
if url != '':
  !mkdir -p /content/drive/MyDrive/project-main/logs/{MODEL}
  !mkdir -p /content/zips/
  if "drive.google.com" in url:
    !gdown $url --fuzzy -O $modelname_path
  elif "mega.nz" in url:
    m = Mega()
    m.download_url(url, '/content/zips')
  else:
    !wget $url -O /content/zips/{MODELZIP}
  for filename in os.listdir("/content/zips"):
    if filename.endswith(".zip"):
      zip_file = os.path.join("/content/zips", filename)
      shutil.unpack_archive(zip_file, "/content/unzips", 'zip')
  #Move model into logs folder
  for root, dirs, files in os.walk('/content/unzips'):
    for file in files:
      if "G_" in file:
        MODELEPOCH = file.split("G_")[1].split(".")[0]
    if MODELEPOCH == '':
      MODELEPOCH = '404'
    for file in files:
      file_path = os.path.join(root, file)
      if file.endswith(".npy") or file.endswith(".index"):
        !mv {file_path} /content/drive/MyDrive/project-main/assets/logs/{MODEL}/
      elif "G_" not in file and "D_" not in file and file.endswith(".pth"):
        !mv {file_path} /content/drive/MyDrive/project-main/assets/weights/{MODEL}.pth


  clear_output()
  display(success)
else:
  print("URL cannot be left empty. If you don't want to download a model now, just skip this step.")

#Training

In [ ]:
#@title 1.Preprocess Data
%cd /content/drive/MyDrive/project-main
model_name = 'My-Voice' #@param {type:"string"}
#@markdown <small> Enter the path to your dataset folder (a folder with audios of the vocals you will train on), or if you want just upload the audios using the File Manager into the 'dataset' folder.
dataset_folder = '/content/drive/MyDrive/dataset' #@param {type:"string"}
while len(os.listdir(dataset_folder)) < 1:
    input("Your dataset folder is empty.")
!mkdir -p ./logs/{model_name}
with open(f'./logs/{model_name}/preprocess.log','w') as f:
    print("Starting...")
!python infer/modules/train/preprocess.py {dataset_folder} 40000 2 ./logs/{model_name} False 3.0 > /dev/null 2>&1
with open(f'./logs/{model_name}/preprocess.log','r') as f:
    if 'end preprocess' in f.read():
        clear_output()
        display(Button(description="\u2714 Success", button_style="success"))
    else:
        print("Error preprocessing data... Make sure your dataset folder is correct.")

In [ ]:
#@title 2.Extract Features
f0method = "rmvpe_gpu" # @param ["pm", "harvest", "rmvpe", "rmvpe_gpu"]
%cd /content/drive/MyDrive/project-main
with open(f'./logs/{model_name}/extract_f0_feature.log','w') as f:
    print("Starting...")
if f0method != "rmvpe_gpu":
    !python infer/modules/train/extract/extract_f0_print.py ./logs/{model_name} 2 {f0method}
else:
    !python infer/modules/train/extract/extract_f0_rmvpe.py 1 0 0 ./logs/{model_name} True
!python infer/modules/train/extract_feature_print.py cuda:0 1 0 0 ./logs/{model_name} v2
with open(f'./logs/{model_name}/extract_f0_feature.log','r') as f:
    if 'all-feature-done' in f.read():
        clear_output()
        display(Button(description="\u2714 Success", button_style="success"))
    else:
        print("Error preprocessing data... Make sure your data was preprocessed.")

In [ ]:
#@title 3.Train Index
import numpy as np
import faiss
%cd /content/drive/MyDrive/project-main
def train_index(exp_dir1, version19):
    exp_dir = "logs/%s" % (exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if not os.path.exists(feature_dir):
        return "请先进行特征提取!"
    listdir_res = list(os.listdir(feature_dir))
    if len(listdir_res) == 0:
        return "请先进行特征提取！"
    infos = []
    npys = []
    for name in sorted(listdir_res):
        phone = np.load("%s/%s" % (feature_dir, name))
        npys.append(phone)
    big_npy = np.concatenate(npys, 0)
    big_npy_idx = np.arange(big_npy.shape[0])
    np.random.shuffle(big_npy_idx)
    big_npy = big_npy[big_npy_idx]
    if big_npy.shape[0] > 2e5:
        infos.append("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
        yield "\n".join(infos)
        try:
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=10000,
                    verbose=True,
                    batch_size=256 * config.n_cpu,
                    compute_labels=False,
                    init="random",
                )
                .fit(big_npy)
                .cluster_centers_
            )
        except:
            info = traceback.format_exc()
            logger.info(info)
            infos.append(info)
            yield "\n".join(infos)

    np.save("%s/total_fea.npy" % exp_dir, big_npy)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    infos.append("%s,%s" % (big_npy.shape, n_ivf))
    yield "\n".join(infos)
    index = faiss.index_factory(256 if version19 == "v1" else 768, "IVF%s,Flat" % n_ivf)
    infos.append("training")
    yield "\n".join(infos)
    index_ivf = faiss.extract_index_ivf(index)  #
    index_ivf.nprobe = 1
    index.train(big_npy)
    faiss.write_index(
        index,
        "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )

    infos.append("adding")
    yield "\n".join(infos)
    batch_size_add = 8192
    for i in range(0, big_npy.shape[0], batch_size_add):
        index.add(big_npy[i : i + batch_size_add])
    faiss.write_index(
        index,
        "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    infos.append(
        "成功构建索引，added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )

training_log = train_index(model_name, 'v2')

for line in training_log:
    print(line)
    if 'adding' in line:
        clear_output()
        display(Button(description="\u2714 Success", button_style="success"))

In [ ]:
 #@title 4.Train Model
#@markdown don't change this!
ngrok_authtoken = "2VqrgGnzNs2bSC0racuiPMMq52U_2sW8K1uh5KTb7rkpNrT7H"#@param {type:"string"}
!ngrok config add-authtoken {ngrok_authtoken}
#%cd /content/drive/MyDrive/project-main
from random import shuffle
import json
import os
import pathlib
from subprocess import Popen, PIPE, STDOUT
from pyngrok import ngrok
now_dir=os.getcwd()
#@markdown <small> Enter the name of your model again. It must be the same you chose before.
model_name = 'My-Voice'#@param {type:"string"}
#@markdown <small> Choose how often to save the model and how much training you want it to have.
save_frequency = 50 # @param {type:"slider", min:5, max:50, step:5}
epochs = 400 # @param {type:"slider", min:10, max:1000, step:10}
#@markdown <small> ONLY cache datasets under 10 minutes long. Otherwise leave this unchecked.
cache = True #@param {type:"boolean"}
# Remove the logging setup

def click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
):
    # 生成filelist
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))

    # Replace logger.debug, logger.info with print statements
    print("Write filelist done")
    print("Use gpus:", str(gpus16))
    if pretrained_G14 == "":
        print("No pretrained Generator")
    if pretrained_D15 == "":
        print("No pretrained Discriminator")
    if version19 == "v1" or sr2 == "40k":
        config_path = "configs/v1/%s.json" % sr2
    else:
        config_path = "configs/v2/%s.json" % sr2
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    cmd = (
        'python infer/modules/train/train.py -e "%s" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s'
        % (
            exp_dir1,
            sr2,
            1 if if_f0_3 else 0,
            batch_size12,
            gpus16,
            total_epoch11,
            save_epoch10,
            "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
            "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
            1 if if_save_latest13 == True else 0,
            1 if if_cache_gpu17 == True else 0,
            1 if if_save_every_weights18 == True else 0,
            version19,
        )
    )
    # Use PIPE to capture the output and error streams
    p = Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=STDOUT, bufsize=1, universal_newlines=True)

    # Print the command's output as it runs
    for line in p.stdout:
        print(line.strip())

    # Wait for the process to finish
    p.wait()
    return "训练结束, 您可查看控制台训练日志或实验文件夹下的train.log"
%load_ext tensorboard
%tensorboard --logdir ./logs --port=8888
print("Tensorboard NGROK URL:",end="")
ngrok_url = ngrok.connect(8888)
print(ngrok_url)
try:
    training_log = click_train(
        model_name,
        '40k',
        True,
        0,
        save_frequency,
        epochs,
        12,
        True,
        'assets/pretrained_v2/f0G40k.pth',
        'assets/pretrained_v2/f0D40k.pth',
        0,
        cache,
        True,
        'v2',
    )
    print(training_log)
except:
    ngrok.kill()

# INFERENCE

In [ ]:
#@title Inference
%cd /content/drive/MyDrive/Retrieval-based-Voice-Conversion-WebUI
model_name = ""#@param {type:"string"}
transpose = "0 "#@param {type:"string"}
index_path = ""#@param {type:"string"}
f0_method = "rmvpe" # @param ["rmvpe", "pm", "harvest"]
input_path = ""#@param {type:"string"}
opt_path = ""#@param {type:"string"}
index_rate = "0.6" #@param {type:"string"}
volume_normalization = 0 # @param {type:"slider", min:0, max:1, step:0.01}
consonant_protection = 0 # @param {type:"slider", min:0, max:1, step:0.01}

!rm $opt_path
!python tools/infer_cli.py --f0up_key $transpose \
--input_path $input_path \
--index_path $index_path \
--f0method $f0_method \
--opt_path $opt_path \
--model_name $model_name \
--index_rate $index_rate \
--device cuda:0 \
--is_half True \
--filter_radius 3 \
--resample_sr 0 \
--rms_mix_rate $volume_normalization \
--protect $consonant_protection

import IPython.display as ipd
ipd.clear_output()
ipd.Audio(opt_path)

# (Run webUI)

In [ ]:
#@title OR Open the GUI (Banned for free Colab Notebooks)
#@markdown <small> Enter your ngrok authtoken to open tensorboard. Get one here: https://dashboard.ngrok.com/get-started/your-authtoken
ngrok_authtoken = "2VqrgGnzNs2bSC0racuiPMMq52U_2sW8K1uh5KTb7rkpNrT7H"#@param {type:"string"}
!ngrok config add-authtoken {ngrok_authtoken}
if not 'installed' in locals():
    %cd /content
    from google.colab import drive
    drive.mount('/content/drive')
    from IPython.display import clear_output
    from ipywidgets import Button
    import os
    if not os.path.exists('/content/drive'):
        print("Your drive is not mounted. Creating Fake Drive.")
        os.makedirs('/content/drive/MyDrive')
    if not os.path.exists('/content/drive/MyDrive/project-main'):
        !wget https://huggingface.co/{source}/project/resolve/main/project-main.zip -O '/content/project-main.zip' && unzip -n 'project-main.zip' -d /content/drive/MyDrive
    !cd '/content/drive/MyDrive/project-main' && python download_files.py && pip install -r 'requirements.txt'
    !rm /content/project-main.zip
    !rm -r /content/sample_data
    !mkdir -p /content/dataset
    !pip install pyngrok
    !pip install tensorflow==2.12.0
    clear_output()
    Button(description="\u2714 Success", button_style="success")
    installed = True
tensorboard = True #@param {type:"boolean"}
%cd /content/drive/MyDrive/project-main
from pyngrok import ngrok
ngrok.kill()
if tensorboard:
    %load_ext tensorboard
    %tensorboard --logdir ./logs --port=8888
    print("Tensorboard NGROK URL:",end="")
    ngrok_url = ngrok.connect(8888)
    print(ngrok_url)
    !python app.py --colab

# <center> Confused? Yes Me Too<br />